In [5]:
from langchain.agents import initialize_agent, AgentType
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [4]:
# load environment variables 
import os
from dotenv import dotenv_values, load_dotenv
load_dotenv()
openai_api_key = os.environ['OPENAI_API_KEY']

In [6]:
model = "gpt-3.5-turbo"
temperature = 0
llm = ChatOpenAI(model=model, temperature=temperature, openai_api_key=openai_api_key

In [17]:
def triage_biopsy(npasses: int, ncores: int = 0, core1_l: float = None, core1_cortex: bool= None, core2_l: float= None, core2_cortex: bool = None,core3_l: float = None, core3_cortex: bool = None, core4_l: float = None, core4_cortex: bool = None, core5_l: float = None, core5_cortex: bool = None):
    max_passes = True if npasses >=5 else False
    diagnostic_core = None
    if core1_l >= 1.4 and core1_cortex: diagnostic_core = "Core 1"
    elif  core2_l and core2_l >= 1.4 and core2_cortex: diagnostic_core = "Core 2"
    elif  core3_l and core3_l >= 1.4 and core3_cortex: diagnostic_core = "Core 3"
    elif  core4_l and core4_l >= 1.4 and core4_cortex: diagnostic_core = "Core 4"
    elif  core5_l and core5_l >= 1.4 and core5_cortex: diagnostic_core = "Core 5"
    else:
        if max_passes: 
            diagnostic_core = "Not ideal but use the longest core to send for diagnostic"
        else:
            diagnostic_core = "Recommend another pass to obtain diagnostic core"
    return diagnostic_core


In [18]:
triage_biopsy(npasses=4, ncores=3, core1_l=1.4, core1_cortex=True)

'Core 1'

In [19]:
triage_biopsy(npasses=4, ncores=3, core1_l=1.3, core1_cortex=True)


'Recommend another pass to obtain diagnostic core'

In [7]:
def calcGFR(creatinine: float, age: float, gender: str):
    gender = gender.lower()
    A = 0.7 if (gender == "female") else 0.9
    B = None
    if (gender == "female" and creatinine <= 0.7): B = -0.241
    elif (gender == "female" and creatinine > 0.7): B = -1.2
    elif (gender == "male" and creatinine <= 0.9): B = -0.302
    elif (gender == "male" and creatinine > 0.9): B = -1.2
    else: raise ValueError
    term1 = 142 * (creatinine / A) ** B
    term2 = 0.9938 ** age
    term3 = 1.0 if gender == "Male" else 1.012
    return term1 * term2 * term3


In [8]:
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class CalculatorInput(BaseModel):
    creatinine: float = Field()
    age: float = Field()
    gender: str = Field()

tools = [
    StructuredTool(
        name = "calcGFR",
        func=calcGFR,
        description="Use this tool to calculate estimated gromeluar filtration rate (GFR) based on patient' serum creatinine, gender (male or female), and age (years)",
        args_schema=CalculatorInput
    )
]

In [9]:
# AgentType = STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION | OPENAI_FUNCTIONS | OPENAI_MULTI_FUNCTIONS
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [10]:
agent.run("A 40 year old male has a serum creatinine of 1.0 mg/dL. What's his estimated glomerular filtration rate?")



> Entering new AgentExecutor chain...

Invoking: `calcGFR` with `{'creatinine': 1.0, 'age': 40, 'gender': 'male'}`


98.7460118977834The estimated glomerular filtration rate (GFR) for a 40-year-old male with a serum creatinine of 1.0 mg/dL is approximately 98.75 mL/min/1.73m².

> Finished chain.


'The estimated glomerular filtration rate (GFR) for a 40-year-old male with a serum creatinine of 1.0 mg/dL is approximately 98.75 mL/min/1.73m².'